In [218]:
import pandas as pd
import numpy as np

In [219]:
df1 = pd.read_excel(
     '../ectas_edm_vitals.xlsx',
     engine='openpyxl', sheet_name = 'edm_vitals'
)

In [220]:
df1

,pt_identifier,screen,date,time,n_query,result
0,H1000412,VITALS,31/12/20,2359,N.000260.1,36.9
1,H1000412,VITALS,31/12/20,2359,N.000280.1,88
2,H1000412,VITALS,31/12/20,2359,N.000290.1,17
3,H1000412,VITALS,31/12/20,2359,N.000300.1,116/84
4,H1000412,VITALS,31/12/20,2359,N.000320.1,97
...,...,...,...,...,...,...
74438,Z98294,VITALS,07/12/20,509,N.000300.1,158/89
74439,Z98294,VITALS,07/12/20,813,N.000260.1,36.4
74440,Z98294,VITALS,07/12/20,813,N.000280.1,72
74441,Z98294,VITALS,07/12/20,813,N.000290.1,20


In [221]:
del df1['screen']


In [222]:
rows = len(df1.index)
df1

,pt_identifier,date,time,n_query,result
0,H1000412,31/12/20,2359,N.000260.1,36.9
1,H1000412,31/12/20,2359,N.000280.1,88
2,H1000412,31/12/20,2359,N.000290.1,17
3,H1000412,31/12/20,2359,N.000300.1,116/84
4,H1000412,31/12/20,2359,N.000320.1,97
...,...,...,...,...,...
74438,Z98294,07/12/20,509,N.000300.1,158/89
74439,Z98294,07/12/20,813,N.000260.1,36.4
74440,Z98294,07/12/20,813,N.000280.1,72
74441,Z98294,07/12/20,813,N.000290.1,20


In [223]:
df2 = pd.DataFrame(data=[], columns= ['pt_identifier', 'Date', 'Time', 'BP1', 'BP2', 'Pulse', 'Temp', 'RR', 'O2Sat'])

In [224]:
df2

,pt_identifier,Date,Time,BP1,BP2,Pulse,Temp,RR,O2Sat


In [225]:
def count_consecutive(df1, i, last_idx):
    count = 1
    
    while np.sum(df1.iloc[i][0:3] == df1.iloc[i + 1][0:3]) == 3:
        count += 1
        i += 1
        if i == last_idx:
            break
    
    return count

In [226]:
def find_index(key_list, key):
    idx = -1
    for x in range(len(key_list)):
        if key_list[x] == key:
            idx = x
        
    return idx

In [227]:
def extract_vitals(df1, i, consecutive):
    p_id = df1.iloc[i][0]
    date = df1.iloc[i][1]
    time = df1.iloc[i][2]
       
    key_list  = list(df1.iloc[i:i+consecutive,3])
    value_list = list(df1.iloc[i:i+consecutive,4])
    
    bp_idx = find_index(key_list, 'N.000300.1')
    pulse_idx = find_index(key_list, 'N.000280.1')
    temp_idx = find_index(key_list, 'N.000260.1')
    rr_idx = find_index(key_list, 'N.000290.1')
    o2sat_idx = find_index(key_list, 'N.000320.1')

    
    bp_combined = value_list[bp_idx] if bp_idx != -1 else ''
    
    bp1, bp2 = bp_combined.split('/') if bp_combined != '' else ['','']
    
    pulse = value_list[pulse_idx] if pulse_idx != -1 else ''
    temp = value_list[temp_idx] if temp_idx != -1 else ''
    rr = value_list[rr_idx] if rr_idx != -1 else ''
    o2sat = value_list[o2sat_idx] if o2sat_idx != -1 else ''

    
    p_vitals = [p_id, date, time, bp1, bp2, pulse, temp, rr, o2sat]
    return p_vitals

In [228]:
i = 0
while i < rows:
    consecutive = count_consecutive(df1, i, rows - 1)
    p_vitals = extract_vitals(df1, i, consecutive)
    df2 = df2.append({'pt_identifier': p_vitals[0], 'Date': p_vitals[1],
                      'Time': p_vitals[2], 'BP1': p_vitals[3], 'BP2': p_vitals[4],
                      'Pulse': p_vitals[5], 'Temp': p_vitals[6], 'RR': p_vitals[7],
                      'O2Sat': p_vitals[8]}, ignore_index=True)
    i += consecutive
  
    


In [232]:
df2.iloc[17660:]

,pt_identifier,Date,Time,BP1,BP2,Pulse,Temp,RR,O2Sat
17660,Z959283,29/11/20,2146,99,69,93,,22,99
17661,Z959369,30/01/21,1502,119,76,100,36.9,17,100
17662,Z966479,23/02/21,2,118,54,80,36.4,20,
17663,Z966479,23/02/21,242,128,59,85,,20,
17664,Z967835,25/01/21,1202,125,89,105,36.2,16,99
17665,Z967835,25/01/21,1330,119,84,105,,16,98
17666,Z967835,25/01/21,1452,132,89,98,,18,99
17667,Z9694,25/10/20,338,124,61,79,36.6,14,99
17668,Z9694,25/10/20,402,129,66,88,,14,99
17669,Z977742,26/03/21,1712,120,77,54,37.2,16,98


In [233]:
df2.to_csv('formatted_edm_vitals.csv')


In [134]:
# run time approx 5 minutes for 75,000 rows